
# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

## İş Problemi
Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak seviye tabanlı (level based) yeni müşteri tanımları (persona) oluşturmak ve bu yeni müşteri tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.

**Örnek:**  
Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek kullanıcının ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

## Veri Seti Hikayesi
Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu ürünleri satın alan kullanıcıların bazı demografik bilgilerini barındırmaktadır. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo tekilleştirilmemiştir. Diğer bir ifade ile belirli demografik özelliklere sahip bir kullanıcı birden fazla alışveriş yapmış olabilir.

## Değişkenler
- **PRICE:** Müşterinin harcama tutarı  
- **SOURCE:** Müşterinin bağlandığı cihaz türü  
- **SEX:** Müşterinin cinsiyeti  
- **COUNTRY:** Müşterinin ülkesi  
- **AGE:** Müşterinin yaşı  

## Görevler
1. persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.  
2. Kaç unique SOURCE vardır? Frekansları nedir?  
3. Kaç unique PRICE vardır?  
4. Hangi PRICE’dan kaçar tane satış gerçekleşmiştir?  
5. Hangi ülkeden kaçar tane satış olmuştur?  
6. Ülkelere göre satışlardan toplam ne kadar kazanılmıştır?  
7. SOURCE türlerine göre satış sayıları nedir?  
8. Ülkelere göre PRICE ortalamaları nedir?  
9. SOURCE’lara göre PRICE ortalamaları nedir?  
10. COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?  
11. COUNTRY-SOURCE-SEX-AGE kırılımında ortalama kazanç nedir?  
12. Çıktıyı PRICE’a göre azalan sıralayınız (agg_df olarak kaydediniz).  
13. İndekste yer alan isimleri değişken ismine çeviriniz.  
14. AGE değişkenini kategorik değişkene çeviriniz.  
15. Yeni seviye tabanlı müşterileri (persona) tanımlayınız.  
16. Yeni müşterileri (personaları) segmentlere ayırınız.  
17. Yeni gelen müşterileri sınıflandırıp gelir tahmini yapınız.  


### Öncelikle kullanacağımız kütüphaneleri yükleyelim ve ayarlamaları yapalım:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import warnings
warnings.filterwarnings('ignore')

### SORU1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.

In [ ]:
df = pd.read_csv(r'C:\Users\ceren sahin\PycharmProjects\PYTHON_MIUUL\datasets\persona.csv')

In [ ]:
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [ ]:
df.tail()

,PRICE,SOURCE,SEX,COUNTRY,AGE
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31
4999,29,android,female,bra,31


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [ ]:
df.shape

(5000, 5)

In [ ]:
df.isnull().sum()

PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64

In [ ]:
df.columns

Index(['PRICE', 'SOURCE', 'SEX', 'COUNTRY', 'AGE'], dtype='object')

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.00,34.13,12.46,9.00,29.00,39.00,39.00,59.00
AGE,5000.00,23.58,9.00,15.00,17.00,21.00,27.00,66.00


### SORU2: kaç unique source vardır? frekansları nedir?

In [ ]:
df['SOURCE'].nunique()

2

In [ ]:
df['SOURCE'].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

### SORU3: kaç unique price vardır?

In [ ]:
df['PRICE'].nunique()

6

### SORU4: hangi price'dan kaçar tane satış gerçekleşmiştir?

In [ ]:
df['PRICE'].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

### SORU5: hangi ülkeden kaçar tane satış olmuştur?

In [ ]:
df['COUNTRY'].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

### SORU6: ülkelere göre satışlardan toplam ne kadar kazanılmıştır?

In [ ]:
df.groupby('COUNTRY').agg({'PRICE':'sum'})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


### SORU7: source türlerine göre satış sayıları nedir?

In [ ]:
df['SOURCE'].value_counts()     #böyle de bulabiliriz

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

In [ ]:
df.groupby('SOURCE').agg({'PRICE':'count'})

,PRICE
SOURCE,
android,2974
ios,2026


### SORU8: ülkelere göre price ortalamaları nedir?

In [ ]:
df.groupby('COUNTRY').agg({'PRICE':'mean'})

,PRICE
COUNTRY,
bra,34.33
can,33.61
deu,34.03
fra,33.59
tur,34.79
usa,34.01


### SORU9: source'lara göre price ortalamaları nedir?

In [ ]:
df.groupby('SOURCE').agg({'PRICE':'mean'})

,PRICE
SOURCE,
android,34.17
ios,34.07


### SORU10: country-source kırılımında price ortalamaları nedir?

In [ ]:
df.groupby(['COUNTRY','SOURCE']).agg({'PRICE':'mean'})

PRICE
COUNTRY SOURCE        
bra     android  34.39
        ios      34.22
can     android  33.33
        ios      33.95
deu     android  33.87
        ios      34.27
fra     android  34.31
        ios      32.78
tur     android  36.23
        ios      33.27
usa     android  33.76
        ios      34.37

### SORU11: country-source-sex-age kırılımında ortalama kazanç nedir?

In [ ]:
df.groupby(['COUNTRY','SOURCE','SEX','AGE']).agg({'PRICE':'mean'})

PRICE
COUNTRY SOURCE  SEX    AGE       
bra     android female 15   38.71
                       16   35.94
                       17   35.67
                       18   32.26
                       19   35.21
...                           ...
usa     ios     male   42   30.25
                       50   39.00
                       53   34.00
                       55   29.00
                       59   46.50

[348 rows x 1 columns]

### SORU12: önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde price'a göre uygulayınız.  

  Çıktıyı agg_df olarak kaydediniz.

In [ ]:
agg_df = df.groupby(['COUNTRY','SOURCE','SEX','AGE']).agg({'PRICE':'mean'}).sort_values('PRICE', ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.00
usa,android,male,36,59.00
fra,android,female,24,59.00
usa,ios,male,32,54.00
deu,android,female,36,49.00


### SORU13: indekste yer alan isimleri değişken ismine çeviriniz.  

  Önceki sorunun çıktısında yer alan price dışındaki tüm değişkenler index isimleridir. bu isimleri değişken ismine çeviriniz.

In [ ]:
agg_df.reset_index(inplace=True)
agg_df.head(10)

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.00
1,usa,android,male,36,59.00
2,fra,android,female,24,59.00
3,usa,ios,male,32,54.00
4,deu,android,female,36,49.00
5,deu,android,female,51,49.00
6,deu,ios,male,20,49.00
7,usa,android,female,47,49.00
8,fra,ios,male,20,49.00
9,usa,ios,male,59,46.50


### SORU15: age değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.  

  Age sayısal değişkeni kategorik değişkene çeviriniz.  
  Aralıkları ikna edici şekilde oluşturunuz.  
  Örneğin: '0-18','19-23','24-30','31-40','41-70'  

In [ ]:
bins=[0,18,23,30,40,70]
labels=['0-18','19-23','24-30','31-40','41-70']
agg_df['AGE_CAT'] = pd.cut(agg_df['AGE'], bins=bins, labels=labels, include_lowest=True)          #agg_df[] şeklinde yazmayı unutma!
agg_df.head(20)

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.00,41-70
1,usa,android,male,36,59.00,31-40
2,fra,android,female,24,59.00,24-30
3,usa,ios,male,32,54.00,31-40
4,deu,android,female,36,49.00,31-40
5,deu,android,female,51,49.00,41-70
6,deu,ios,male,20,49.00,19-23
7,usa,android,female,47,49.00,41-70
8,fra,ios,male,20,49.00,19-23
9,usa,ios,male,59,46.50,41-70


### SORU16: yeni seviye tabanlı müşterileri (persona) tanımlayınız.  

  Yeni seviye tabanlı müşterileri (persona) tanımlayınız ve veri setine değişken olarak ekleyiniz.  
  Yeni eklenecek değişkenin adı: customers_level_based  
  Önceki soruda elde edeceğiniz çıktıdaki gözlemleri bir araya getirerek customers_level_based değişkenini oluşturmanız gerekmektedir.  

In [ ]:
agg_df['customers_level_based'] = agg_df[['COUNTRY','SOURCE','SEX','AGE_CAT']].agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df.head(20)

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
0,bra,android,male,46,59.00,41-70,BRA_ANDROID_MALE_41-70
1,usa,android,male,36,59.00,31-40,USA_ANDROID_MALE_31-40
2,fra,android,female,24,59.00,24-30,FRA_ANDROID_FEMALE_24-30
3,usa,ios,male,32,54.00,31-40,USA_IOS_MALE_31-40
4,deu,android,female,36,49.00,31-40,DEU_ANDROID_FEMALE_31-40
5,deu,android,female,51,49.00,41-70,DEU_ANDROID_FEMALE_41-70
6,deu,ios,male,20,49.00,19-23,DEU_IOS_MALE_19-23
7,usa,android,female,47,49.00,41-70,USA_ANDROID_FEMALE_41-70
8,fra,ios,male,20,49.00,19-23,FRA_IOS_MALE_19-23
9,usa,ios,male,59,46.50,41-70,USA_IOS_MALE_41-70


In [ ]:
agg_df = agg_df[['customers_level_based','PRICE']]     #sadece bu iki değişkene ihtiyacımız var, gereksiz değişkenleri çıkaralım
agg_df.head(10)

,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41-70,59.00
1,USA_ANDROID_MALE_31-40,59.00
2,FRA_ANDROID_FEMALE_24-30,59.00
3,USA_IOS_MALE_31-40,54.00
4,DEU_ANDROID_FEMALE_31-40,49.00
5,DEU_ANDROID_FEMALE_41-70,49.00
6,DEU_IOS_MALE_19-23,49.00
7,USA_ANDROID_FEMALE_41-70,49.00
8,FRA_IOS_MALE_19-23,49.00
9,USA_IOS_MALE_41-70,46.50


In [ ]:
agg_df['customers_level_based'].value_counts()     #tekil değil henüz tekilleştirmemiz lazım

customers_level_based
BRA_ANDROID_MALE_24-30      7
USA_ANDROID_MALE_41-70      7
USA_IOS_FEMALE_24-30        7
BRA_ANDROID_FEMALE_24-30    7
USA_ANDROID_MALE_24-30      7
                           ..
TUR_ANDROID_MALE_41-70      1
CAN_ANDROID_MALE_19-23      1
TUR_IOS_MALE_31-40          1
TUR_IOS_MALE_24-30          1
CAN_ANDROID_FEMALE_24-30    1
Name: count, Length: 109, dtype: int64

In [ ]:
agg_df = agg_df.groupby('customers_level_based').agg({'PRICE':'mean'}).reset_index()     #burada tekilleştirdik
agg_df.head(10)

,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0-18,35.65
1,BRA_ANDROID_FEMALE_19-23,34.08
2,BRA_ANDROID_FEMALE_24-30,33.86
3,BRA_ANDROID_FEMALE_31-40,34.90
4,BRA_ANDROID_FEMALE_41-70,36.74
5,BRA_ANDROID_MALE_0-18,34.81
6,BRA_ANDROID_MALE_19-23,31.67
7,BRA_ANDROID_MALE_24-30,33.41
8,BRA_ANDROID_MALE_31-40,34.33
9,BRA_ANDROID_MALE_41-70,40.04


### SORU17: yeni müşterileri (personaları) segmentlere ayırınız.  

  Yeni müşterileri (örnek: USA_ANDROID_MALE_0-18) price'a göre 4 segmente ayırınız.  
  Segmentleri SEGMENT isimlendirmesi ile değişken olarak agg_df'e ekleyiniz.  
  Segmentleri betimleyiniz. (segmentlere göre groupby yapıp price mean, max, sum'larını alınız.)  

In [ ]:
agg_df['SEGMENT'] = pd.qcut(agg_df['PRICE'], 4, labels=['D','C','B','A'])
agg_df.groupby('SEGMENT').agg({'PRICE':['mean','max','sum']})

PRICE              
         mean   max     sum
SEGMENT                    
D       29.21 32.33  817.79
C       33.51 34.08  904.76
B       35.00 36.00  944.99
A       38.69 45.43 1044.66

### SORU18: yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.  

  33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?  
  35 yaşında OIS kullanan bir Fransız kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?  

In [ ]:
new_user='TUR_ANDROID_FEMALE_31-40'
agg_df[agg_df['customers_level_based'] == new_user]          #41.83 dolar kazandırması beklenir.

,customers_level_based,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31-40,41.83,A


In [ ]:
new_user='FRA_IOS_FEMALE_31-40'
agg_df[agg_df['customers_level_based'] == new_user]          #32.82 dolar kazandırması beklenir.

,customers_level_based,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31-40,32.82,C
